## Czym są i do czego służą potoki?

W większości projektów uczenia maszynowego dane, z którymi przyjdzie nam pracować będą wymagały obróbki (nie będą w formacie idealnym do stworzenia najlepszego modelu ML). Bardzo często konieczne jest wykonanie wielu transformacji naszych danych takich, jak kodowanie zmiennych kategorycznych (encoding categorical variables), skalowanie cech (feature scaling) czy normalizacja. Biblioteka Scikit-learn posiada wbudowane funkcje dla większości powszechnie używanych przekształceń.

Jednak w typowym projekcie polegającym na budowie modelu ML trzeba bedzie zastosować transformacje danych co najmniej dwa razy - raz podczas trenowania modeli i ponownie dla nowych danych (testowanie modelu). Oczywiście możliwe jest napisanie funkcji, które wykonają wszystkie przekształcenia, ale wymaga to poświęcenia czasu na ich zdefiniowanie, a następnie za każdym razem trzeba je wywoływać dla naszych danych i OSOBNO wywołać model. Narzędziem zdecydowanie upraszczającym ten proces są właśnie **potoki** (ang. pipelines). 

**Zalety potoków:**

- czyściejszy kod (przepływ danych i wykonywane operacje są znacznie łatwiejsze do zrozumienia)
- mniej błędów (bugów) i łatwiejsza ich naprawa
- potoki wymuszają określoną kolejność operacji w naszym projekcie
- łatwiejsze zastosowanie przekształceń dla nowych danych -> łatwiejsze wprowadzenie kodu na produkcję
- więcej sposobów na walidację modelu (np. walidacja krzyżowa z użyciem potoków jest łatwiejsza do przeprowadzenia).

**Podsumowanie:**

Potoki pozwalają wykonać obróbkę danych w dużo bardziej przejrzysty sposób, bez konieczności pisania wielu linii kodu (jest to bardzo przydatne, gdy najpierw obrabiamy dane treningowe, a następnie tę samą obróbkę musimy zastosować do zbioru testowego). Inaczej, potoki obejmują etapy wstępnego przetwarzania i modelowania danych oraz pozwalają dobrać najlepsze możliwe transformacje naszych danych. Ponadto, możliwe jest zastosowanie ich do walidacji i wyboru najlepszego modelu oraz, w połączeniu z *siatką przeszukiwań* (ang. GridSearch), do dostrojenia hiperparametrów naszego modelu.

## Etapy tworzenia potoków

1. Pierwszym etapem jest zdefiniowanie wszystkich rodzajów transformacji. Według konwencji tworzymy transformatory (ang. transformers) w zależności od typu danych, który mają modyfikować (jeden dla zmiennych kategorycznych, jeden dla numerycznych.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# tworzymy listy kolumn numerycznych i kategorycznych
# (na tym etapie zb. treningowy nie powinien już zawierać 'targetu' (y_train), bo jego nie chcemy modyfikować)
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

# tworzymy transformatory
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

2. Kolejnym etapem jest zastosowanie utworzonych transformacji na wskazanych kolumnach DF za pomocą metody *ColumnTransformer*.

    *ColumnTransformer* pozwala przypisać konkretne transformacje do konkretnych kolumn oraz utworzyć jedną "przestrzeń cech" (ang. feature space) po transformacjach.

In [ ]:
from sklearn.compose import ColumnTransformer

data_transformer = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features)
])

    OPCJONALNIE: W zależności od rodzaju modelu, który budujemy, może być również konieczna redukcja wymiarów naszej macierzy danych, np. za pomocą PCA. Dlatego poniższy kod pokazuje, jak zastosować PCA z użyciem potoków w nawiązaniu do powyższych punktów.

In [ ]:
# Tworzenie potoku, które najpierw transformuje dane (jw.), a następnie stosuje PCA
preprocessor = Pipeline(steps=[
    ('data_transformer', data_transformer),
    ('reduce_dim', PCA())
])

3. Dodawanie algorytmu do potoku (np. klasyfikatora)

    Kolejnym krokiem jest połączenie preprocessora dla danych utworzonego powyżej z algorytmem ML.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Teraz możemy w prosty sposób uruchomić "trening" na naszych surowych danych, a utworzony powyżej "classifier"
# zadba o ich preprocessing
rf.fit(X_train, y_train)

# Następnie predykcja
y_pred = rf.predict(X_test)

Jeśli chcemy wytrenować pojedynczy model, to możemy zakończyć budowę modelu na wynikach predykcji z pkt. 3 (jw.). Możemy również wykorzystać potoki do wyboru najlepszego spośród modeli (jn.)

4. Wybór najlepszego modelu z domyślnymi parametrami

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

Powyższy kod zwróci jakość każdego z powyższych klasyfikatorów w zbliżonej formie do poniższej:

<img src="Images/img_83.jpg">

Potoki mogą również posłużyć do wyboru najlepszych hiperparametrów dla naszego modelu (jn.).

5. Wybór hiperparametrów

    Pierwszym krokiem jest utworzenie siatki hiperparametrów dla naszego modelu (param_grid). Ważną rzeczą, o której należy tutaj pamiętać jest konieczność dodania do domyślnej nazwy danego parametru nazwy naszego utworzonego klasyfikatora, czyli w nawiązaniu do powyższych punktów będzie to przedrostek w postaci *classifier__*.

    Następnie tworzymy obiekt GridSearch do przeszukiwania siatki parametrów. Musi on zawierać oryginalny potok ( *rf* ), aby uwzględnić wszystkie określone wcześniej transformacje. Kiedy teraz wywołujemy "trening"(fit), transformacje zostają zaaplikowane do naszych danych zanim zostanie wywołana walidacja krzyżowa dla siatki przeszukiwań (ang. before a cross-validated grid-search is performed over the parameter grid).

In [ ]:
from sklearn.model_selection import GridSearchCV

# założyliśmy, że najlepszym spośród wszystkich sprawdzonych modeli był ten oparty na RandomForestClassifier (rf)
# a więc poniższe parametry odnoszą się właśnie do niego.
param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']
}

CV = GridSearchCV(rf, param_grid, n_jobs= 1)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

**Atrykuły:**

https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf

https://towardsdatascience.com/are-you-using-pipeline-in-scikit-learn-ac4cd85cb27f

https://queirozf.com/entries/scikit-learn-pipeline-examples